In [1]:
import os, sys
cur_dir = os.getcwd()
basename = os.path.basename(cur_dir)
for _ in range(5):
    if basename != 'OCLog':
        cur_dir = os.path.dirname(cur_dir)
        basename = os.path.basename(cur_dir)
        #print(cur_dir, basename)
    else:
        if cur_dir not in sys.path:
            sys.path.append(cur_dir)
            #print(sys.path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import trange, tqdm, tnrange
from oclog.BGL.bgUKCNum import BGLog, get_embedding_layer
from oclog.openset.boundary_loss import euclidean_metric, BoundaryLoss
from oclog.openset.pretraining import LogLineEncoder, LogSeqEncoder, LogClassifier
# from oclog.openset.openset import OpenSet
from oclog.openset.opensetv6 import OpenSet
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import sklearn.metrics as m
import warnings

In [3]:
#KERNSTOR 1118765464 2005.06.14 R02-M1-N7-C:J09-U01 2005-06-14-09.11.04.290239 R02-M1-N7-C:J09-U01 RAS KERNEL FATAL data storage interrupt
bglog = BGLog(padded_char_len=128, clean_part_1=False, clean_part_2=False, clean_time_1=False, clean_part_4=False, 
            clean_time_2=False,clean_part_6=False, debug=True, pkl_file='bgl_ukc_num.pkl',
             tk_file='bgl_tk_num.pkl', save_padded_num_sequences=True, load_from_pkl=False)
train_test = bglog.get_tensor_train_val_test(ablation=500, designated_ukc_cls=5 )
train_data, val_data, test_data = train_test

In [4]:
# train_test = bglog.get_tensor_train_val_test(ablation=500, designated_ukc_cls=5 )
# train_data, val_data, test_data = train_test

ablation set to : 500
log file path found:  True
total number of lines in the log file: 4747963
RAM usage:  39064728
ending logs in memory: 0.8202934265136719
alerts 4747963
unique_alerts: {'FATAL', 'microseconds', 'FAILURE', '0x00544eb8,', 'ERROR', 'single', 'SEVERE', 'Kill', 'WARNING', 'INFO'}
length of list of sequence 148373
elapsed time: 2.2861292362213135
len of whole_text_for_training 4747936
character vocabulary 50
148373
                                                 seq label
0  [[3, 3, 3, 15, 19, 11, 19, 8, 15, 2, 5, 2, 2, ...  INFO
1  [[3, 3, 3, 15, 19, 11, 19, 8, 15, 8, 5, 2, 2, ...  INFO
2  [[3, 3, 3, 15, 19, 11, 19, 8, 19, 2, 5, 2, 2, ...  INFO
3  [[3, 3, 3, 15, 19, 11, 19, 8, 19, 14, 5, 2, 2,...  INFO
4  [[3, 3, 3, 15, 19, 11, 19, 8, 22, 15, 5, 2, 2,...  INFO
0    114115
1     28439
2      4049
3       902
4       628
5       165
6        75
Name: label, dtype: int64
completed padding sequences in 363.01476979255676 sec
trying to save pickle in : C:\Users\Bhujay_ROG\M

In [6]:
bglog.cleaned_labelled_sequences

[(['111783857020050603r02m1n0cj12u1120050603154250363779r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857020050603r02m1n0cj12u1120050603154250527847r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857020050603r02m1n0cj12u1120050603154250675872r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857020050603r02m1n0cj12u1120050603154250823719r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857020050603r02m1n0cj12u1120050603154250982731r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857120050603r02m1n0cj12u1120050603154251131467r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857120050603r02m1n0cj12u1120050603154251293532r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857120050603r02m1n0cj12u1120050603154251428563r02m1n0cj12u11raskernelinfoinstructioncacheparityerrorcorrected',
   '111783857120050603r0

In [8]:
fstr='111795305820050604r25m0n0ij18u0120050604233058910314r25m0n0ij18u01rasappfatalcioderrorloadinghomestreitzsrcddcmd1111binddcmdbglvinvalidormissingprogramimagenosuchfileordirectory'
print(len(fstr))

177


In [12]:
line_encoder = LogLineEncoder(bglog, chars_in_line=128)
logSeqencer =  LogSeqEncoder(line_in_seq=32, dense_neurons=10)
ptmodel = LogClassifier(line_encoder=line_encoder, seq_encoder=logSeqencer, num_classes=5)
ptmodel.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = ptmodel.fit(train_data, validation_data=val_data, epochs=10)    

vocab_size: 50
Epoch 1/10
62/62 [==============================] - 12s 178ms/step - loss: 0.9073 - accuracy: 0.6326 - precision_3: 0.8364 - recall_3: 0.4355 - val_loss: 0.2495 - val_accuracy: 0.9241 - val_precision_3: 0.9241 - val_recall_3: 0.9241
Epoch 2/10
62/62 [==============================] - 11s 176ms/step - loss: 0.1939 - accuracy: 0.9289 - precision_3: 0.9335 - recall_3: 0.9269 - val_loss: 0.0885 - val_accuracy: 0.9732 - val_precision_3: 0.9776 - val_recall_3: 0.9732
Epoch 3/10
62/62 [==============================] - 11s 175ms/step - loss: 0.0847 - accuracy: 0.9738 - precision_3: 0.9738 - recall_3: 0.9728 - val_loss: 0.0998 - val_accuracy: 0.9688 - val_precision_3: 0.9686 - val_recall_3: 0.9643
Epoch 4/10
62/62 [==============================] - 11s 176ms/step - loss: 0.0775 - accuracy: 0.9723 - precision_3: 0.9728 - recall_3: 0.9723 - val_loss: 0.1973 - val_accuracy: 0.9107 - val_precision_3: 0.9107 - val_recall_3: 0.9107
Epoch 5/10
62/62 [==============================] - 1

In [16]:
#KERNSTOR 1118765464 2005.06.14 R02-M1-N7-C:J09-U01 2005-06-14-09.11.04.290239 R02-M1-N7-C:J09-U01 RAS KERNEL FATAL data storage interrupt
padded_char_len=176
bglog = BGLog(padded_char_len=padded_char_len, clean_part_1=False, clean_part_2=False, clean_time_1=False, clean_part_4=False, 
            clean_time_2=False,clean_part_6=False, debug=True, pkl_file='bgl_ukc_176.pkl',
             tk_file='bgl_tk_176.pkl', save_padded_num_sequences=True, load_from_pkl=False)
train_test = bglog.get_tensor_train_val_test(ablation=1000, designated_ukc_cls=5 )
train_data, val_data, test_data = train_test

ablation set to : 1000
log file path found:  True
total number of lines in the log file: 4747963
RAM usage:  39064728
ending logs in memory: 0.9247303009033203
alerts 4747963
unique_alerts: {'FATAL', 'microseconds', 'FAILURE', '0x00544eb8,', 'ERROR', 'single', 'SEVERE', 'Kill', 'WARNING', 'INFO'}
length of list of sequence 148373
elapsed time: 2.072495222091675
len of whole_text_for_training 4747936
character vocabulary 50
148373
                                                 seq label
0  [[3, 3, 3, 15, 19, 11, 19, 8, 15, 2, 5, 2, 2, ...  INFO
1  [[3, 3, 3, 15, 19, 11, 19, 8, 15, 8, 5, 2, 2, ...  INFO
2  [[3, 3, 3, 15, 19, 11, 19, 8, 19, 2, 5, 2, 2, ...  INFO
3  [[3, 3, 3, 15, 19, 11, 19, 8, 19, 14, 5, 2, 2,...  INFO
4  [[3, 3, 3, 15, 19, 11, 19, 8, 22, 15, 5, 2, 2,...  INFO
0    114115
1     28439
2      4049
3       902
4       628
5       165
6        75
Name: label, dtype: int64
completed padding sequences in 365.53554916381836 sec
trying to save pickle in : C:\Users\Bhujay_ROG\M

In [19]:

line_encoder = LogLineEncoder(bglog, chars_in_line=padded_char_len)
logSeqencer =  LogSeqEncoder(line_in_seq=32, dense_neurons=12)
ptmodel = LogClassifier(line_encoder=line_encoder, seq_encoder=logSeqencer, num_classes=4)
ptmodel.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = ptmodel.fit(train_data, validation_data=val_data, epochs=10)

vocab_size: 50
Epoch 1/10
100/100 [==============================] - 25s 247ms/step - loss: 0.5408 - accuracy: 0.7344 - precision_7: 0.8384 - recall_7: 0.6616 - val_loss: 0.0978 - val_accuracy: 0.9948 - val_precision_7: 0.9948 - val_recall_7: 0.9922
Epoch 2/10
100/100 [==============================] - 25s 246ms/step - loss: 0.0768 - accuracy: 0.9762 - precision_7: 0.9765 - recall_7: 0.9759 - val_loss: 0.0138 - val_accuracy: 0.9974 - val_precision_7: 0.9974 - val_recall_7: 0.9974
Epoch 3/10
100/100 [==============================] - 25s 247ms/step - loss: 0.0295 - accuracy: 0.9897 - precision_7: 0.9897 - recall_7: 0.9897 - val_loss: 0.0406 - val_accuracy: 0.9896 - val_precision_7: 0.9948 - val_recall_7: 0.9896
Epoch 4/10
100/100 [==============================] - 25s 252ms/step - loss: 0.0279 - accuracy: 0.9912 - precision_7: 0.9916 - recall_7: 0.9912 - val_loss: 0.0076 - val_accuracy: 1.0000 - val_precision_7: 1.0000 - val_recall_7: 1.0000
Epoch 5/10
100/100 [=========================